# Scenario 5

A retail company maintains a product catalog in its data warehouse. Product details such as name, category and price may change over time due to rebranding, category updates or pricing adjustments. To preserve historical data for accurate reporting and trend analysis, the company needs to implement a **Slowly Changing Dimension (SCD) Type 2** mechanism in PySpark, ensuring old records are retained with effective data ranges while new versions are inserted as separate records.

In [0]:
%sql
CREATE TABLE pyspark_cata.source.customers
(
  id STRING,
  email STRING,
  city STRING,
  country STRING,
  modifiedDate TIMESTAMP
)

In [0]:
%sql
INSERT INTO pyspark_cata.source.customers
VALUES
('1', 'john.smith@example.com', 'Seattle', 'USA', current_timestamp()),
('6', 'jane.doe@example.com', 'London', 'UK', current_timestamp())

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
SELECT * FROM pyspark_cata.source.customers

id,email,city,country,modifiedDate
1,john.smith@example.com,New York,USA,2025-08-20T09:02:56.308Z
2,jane.doe@example.com,London,UK,2025-08-20T09:02:56.308Z
3,mike.williams@example.com,Paris,France,2025-08-20T09:02:56.308Z
4,sara.jones@example.com,Tokyo,Japan,2025-08-20T09:02:56.308Z
5,peter.chan@example.com,Sydney,Australia,2025-08-20T09:02:56.308Z
1,john.smith@example.com,Seattle,USA,2025-08-20T09:54:36.074Z
6,jane.doe@example.com,London,UK,2025-08-20T09:54:36.074Z


In [0]:
if spark.catalog.tableExists("pyspark_cata.source.DimCustomers"):
    pass
else:
    spark.sql("""
              CREATE TABLE pyspark_cata.source.DimCustomers
              SELECT *,
                current_timestamp() as startTime,
                CAST('3000-01-01' AS TIMESTAMP) as endTime,
                'Y' as isActive
              FROM pyspark_cata.source.customers
              """)

In [0]:
%sql
SELECT * FROM pyspark_cata.source.DimCustomers

id,email,city,country,modifiedDate,startTime,endTime,isActive
1,john.smith@example.com,New York,USA,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y
2,jane.doe@example.com,London,UK,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y
3,mike.williams@example.com,Paris,France,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y
4,sara.jones@example.com,Tokyo,Japan,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y
5,peter.chan@example.com,Sydney,Australia,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y


## **SCD TYPE-2**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [0]:
df = spark.sql("""
               SELECT * FROM pyspark_cata.source.customers
               """)

df = df.withColumn("dedup", row_number().over(Window.partitionBy('id').orderBy(desc('modifiedDate'))))\
  .drop('dedup')

df = df.filter(col('dedup') == 1)

df.createOrReplaceTempView('srctemp')

df = spark.sql("""
              SELECT *,
                current_timestamp() as startTime,
                CAST('3000-01-01' AS TIMESTAMP) as endTime,
                'Y' as isActive
              FROM srctemp
              """)

df.createOrReplaceTempView('src')

In [0]:
%sql
SELECT * FROM src

id,email,city,country,modifiedDate,startTime,endTime,isActive
1,john.smith@example.com,Seattle,USA,2025-08-20T09:54:36.074Z,2025-08-20T09:59:23.329Z,3000-01-01T00:00:00.000Z,Y
2,jane.doe@example.com,London,UK,2025-08-20T09:02:56.308Z,2025-08-20T09:59:23.329Z,3000-01-01T00:00:00.000Z,Y
3,mike.williams@example.com,Paris,France,2025-08-20T09:02:56.308Z,2025-08-20T09:59:23.329Z,3000-01-01T00:00:00.000Z,Y
4,sara.jones@example.com,Tokyo,Japan,2025-08-20T09:02:56.308Z,2025-08-20T09:59:23.329Z,3000-01-01T00:00:00.000Z,Y
5,peter.chan@example.com,Sydney,Australia,2025-08-20T09:02:56.308Z,2025-08-20T09:59:23.329Z,3000-01-01T00:00:00.000Z,Y
6,jane.doe@example.com,London,UK,2025-08-20T09:54:36.074Z,2025-08-20T09:59:23.329Z,3000-01-01T00:00:00.000Z,Y


### **MERGE-1 Marking the updated records as expired**

In [0]:
%sql
MERGE INTO pyspark_cata.source.DimCustomers as trg
USING src as src
ON trg.id = src.id
AND trg.isActive = 'Y'

WHEN MATCHED AND
src.email <> trg.email
OR src.city <> trg.city
OR src.country <> trg.country
OR src.modifiedDate <> trg.modifiedDate

THEN UPDATE SET
trg.endTime = current_timestamp(),
trg.isActive = 'N'

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


### **MERGE-2 Inserting New And Updated Records**

In [0]:
%sql
MERGE INTO pyspark_cata.source.DimCustomers as trg
USING src as src
ON src.id = trg.id
AND trg.isActive = 'Y'

WHEN NOT MATCHED
THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,0,0,2


In [0]:
%sql
SELECT * FROM pyspark_cata.source.DimCustomers

id,email,city,country,modifiedDate,startTime,endTime,isActive
2,jane.doe@example.com,London,UK,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y
3,mike.williams@example.com,Paris,France,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y
4,sara.jones@example.com,Tokyo,Japan,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y
5,peter.chan@example.com,Sydney,Australia,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,3000-01-01T00:00:00.000Z,Y
1,john.smith@example.com,New York,USA,2025-08-20T09:02:56.308Z,2025-08-20T09:05:31.271Z,2025-08-20T10:00:54.759Z,N
1,john.smith@example.com,Seattle,USA,2025-08-20T09:54:36.074Z,2025-08-20T10:01:32.854Z,3000-01-01T00:00:00.000Z,Y
6,jane.doe@example.com,London,UK,2025-08-20T09:54:36.074Z,2025-08-20T10:01:32.854Z,3000-01-01T00:00:00.000Z,Y
